In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob,os
import sys
import scipy
from importlib import  reload
from time import process_time 
#from libraries.lib_gather_data import get_hhid_FIES

%load_ext autoreload



In [2]:
from shock_libraries import *
from plotting_libraries import *
from response_libraries import get_response_sp
#
from income_shock_libraries_ps import *
#
from libraries.lib_country_dir import set_directories, load_survey_data, get_places_dict
from libraries.lib_get_hh_savings import get_hh_savings
from libraries.pandas_helper import broadcast_simple

# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

In [3]:
generate_shock_100()

/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/psaylor/Documents/GitHub/covid_phl/income_shock_libraries_ps.py:843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mr_subset.partial_prob[i] = mr_subset.sector_frac[i] * (random.randint(0,50)/100)
/Users/psaylor/Documents/GitHub/covid_phl/income_shock_libraries_ps.py:859: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [4]:
get_shock_stats()

,mean,std_dev
LFS_sector,,
ag,0.016,0.002
mining,0.733,0.089
utilities,0.197,0.000
construction,0.769,0.115
manufacturing,0.753,0.012
wholesale,0.680,0.000
retail,0.333,0.001
transportation,0.927,0.010
information,0.000,0.000


In [5]:
def shock_comparison_table_3method_v2():
    """
    modified from: shock_comparison_table()
    on: 20200420
    
    functionality: 
        - creates table to visualize difference between 'fraction affected' by sector
        - for default, social distancing, and 3rd dimension cases
    
    """
    # define default shock table:
    shock_default = { 'ag':           [  0,  0],
                 'mining':        [  0,  0],
                 'utilities':     [  0,  0],
                 'construction':  [0.5,1.0],
                 'manufacturing': [0.1,1.0],
                 'wholesale':     [0.1,1.0],
                 'retail':        [0.5,1.0],
                 'transportation':[0.5,1.0],
                 'information':   [0.1,1.0],
                 'finance':       [0.1,1.0],
                 'professional_services':[0.1,1.0],
                 'eduhealth':     [0.1,1.0],
                 'food_entertainment':[0.8,1.0],
                 'government':    [  0,  0],
                 'other':         [0.8,1.0]}
    df_shock = pd.DataFrame(data=shock_default).T
    df_shock.columns = ['fa','di']
    df_shock.index.name = 'LFS_sector'

    ## load phillipine country stats # original input
    phi_stats = pd.read_csv('./temp/phi_get_shock_input.csv').set_index('LFS_sector')
    #phi_stats.at['ag', 'mean'] = 0.000

    phi_stats = phi_stats.rename(columns={"mean":"social_dist", "std_dev":"SD_std_dev"}) # rename columns
    phi_stats['default'] = df_shock['fa'] # get default values for table

    # phi_stats['difference'] = phi_stats.model_default - phi_stats.phi_sector_model # removed 20200420 <ps>

    
    # add 3dim stats here
    dim3_stats = pd.read_csv('./temp/phi_shock_3dimv2_stats_20200422.csv').set_index('LFS_sector')
    dim3_stats.at['government', 'mean'] = 0.000
    dim3_stats = dim3_stats.rename(columns={"mean":"Full_model", "std_dev":"Full_std_dev"}) # rename columns

    
    # create final table 
    phi_stats = phi_stats[['default','social_dist','SD_std_dev']]
    dim3_stats = dim3_stats[['Full_model','Full_std_dev']]
    
    phi_stats['Full_model'] = dim3_stats.Full_model
    phi_stats['Full_std_dev'] = dim3_stats.Full_std_dev
    phi_stats['diff(SD - Full)'] = phi_stats.social_dist - phi_stats.Full_model
    final_table = phi_stats
    
    return(final_table)


In [6]:
#dim3_stats

In [7]:
x = shock_comparison_table_3method_v2()
x

,default,social_dist,SD_std_dev,Full_model,Full_std_dev,diff(SD - Full)
LFS_sector,,,,,,
ag,0.0,0.016,0.002,0.016,0.002,0.000
mining,0.0,0.925,0.045,0.733,0.089,0.192
utilities,0.0,0.197,0.000,0.197,0.000,0.000
construction,0.5,0.779,0.117,0.769,0.115,0.010
manufacturing,0.1,0.754,0.011,0.753,0.012,0.001
wholesale,0.1,0.680,0.000,0.680,0.000,0.000
retail,0.5,0.333,0.001,0.333,0.001,0.000
transportation,0.5,0.931,0.010,0.927,0.010,0.004
information,0.1,0.000,0.000,0.000,0.000,0.000


In [8]:
y = x[['default','social_dist','Full_model']]
y

,default,social_dist,Full_model
LFS_sector,,,
ag,0.0,0.016,0.016
mining,0.0,0.925,0.733
utilities,0.0,0.197,0.197
construction,0.5,0.779,0.769
manufacturing,0.1,0.754,0.753
wholesale,0.1,0.680,0.680
retail,0.5,0.333,0.333
transportation,0.5,0.931,0.927
information,0.1,0.000,0.000


In [10]:
y
#df_new = df.rename(columns={'A': 'a'}, index={'ONE': 'one'})
z = y.rename(index={'mining': 'extraction'})

In [11]:
z

,default,social_dist,Full_model
LFS_sector,,,
ag,0.0,0.016,0.016
extraction,0.0,0.925,0.733
utilities,0.0,0.197,0.197
construction,0.5,0.779,0.769
manufacturing,0.1,0.754,0.753
wholesale,0.1,0.680,0.680
retail,0.5,0.333,0.333
transportation,0.5,0.931,0.927
information,0.1,0.000,0.000


In [12]:
z.to_csv('./temp/20200422_model_table.csv')